### Spot Market Data Loading

In [49]:
import ccxt

f = open("./APIKEY.txt", 'r')
lines = f.readlines()
f.close()

api_key = lines[0].strip()
secret  = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key, 
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'spot'
    }
})

In [50]:
import time
import pandas as pd

from stockstats import StockDataFrame as Sdf

def get_historical_data(coin_pair, timeframe):
    """Get Historical data (ohlcv) from a coin_pair
    """
    # optional: exchange.fetch_ohlcv(coin_pair, '1h', since)
    since = binance.parse8601('2021-01-01T00:00:00Z')
    data = binance.fetch_ohlcv(since=since, symbol=coin_pair, timeframe=timeframe, limit=30000)
    # update timestamp to human readable timestamp
    data = [[binance.iso8601(candle[0])] + candle[1:] for candle in data]
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(data, columns=header)
    return df


def create_stock(historical_data):
    """Create StockData from historical data 
    """
    stock  = Sdf.retype(historical_data)
    return stock

In [185]:
import calendar
from datetime import datetime, date, timedelta
import numpy as np

def min_ohlcv(dt, pair, limit):
    # UTC native object
    since = calendar.timegm(dt.utctimetuple())*1000
    ohlcv1 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv2 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv = ohlcv1 + ohlcv2
    return ohlcv

def ohlcv(dt, pair, period='1d'):
    startDate, endDate = dt[0], dt[1]
    print(startDate, endDate)
    ohlcv = []
    limit = 1000
    if period == '1m':
        td = 1
        limit = 1440
    elif period == '1d':
        td = 1
        limit = 365 
    elif period == '1h':
        td = 60
        limit = 16
    elif period == '5m':
        td = 5
        limit = 192
        
    start_dt = datetime.strptime(startDate, "%Y%m%d")
    end_dt = datetime.strptime(endDate, "%Y%m%d")
    # 한번에 fetch 가능한 데이터량 1000개
    while (end_dt - start_dt).days > 0:
        diffDays = (end_dt - start_dt).days
        if period == '1h' and diffDays < 61:
            td = diffDays
        if period == '5m' and diffDays < 6:
            td = diffDays
        since = calendar.timegm(start_dt.utctimetuple())*1000
#         print(datetime.fromtimestamp(float(since)/1000), td)
        if period == '1m':
            ohlcv.extend(min_ohlcv(start_dt, pair, limit))
        else:
            ohlcv.extend(binance.fetch_ohlcv(symbol=pair, timeframe=period, since=since, limit=limit*td))
#         print(limit*td)
        start_dt = start_dt + timedelta(days = td)
        

    df = pd.DataFrame(ohlcv, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    time = [datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
    df_t = pd.DataFrame([[t.timetuple().tm_yday,t.timetuple().tm_wday,t.timetuple().tm_hour] for t in time]
                        , columns = ['yDay', 'wDay', 'Hour'])

    del df ['Time']
#     df['Time'] = [datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
    df['Open'] = df['Open'].astype(np.float64)
    df['High'] = df['High'].astype(np.float64)
    df['Low'] = df['Low'].astype(np.float64)
    df['Close'] = df['Close'].astype(np.float64)
    df['Volume'] = df['Volume'].astype(np.float64)

    df = pd.concat([df_t, df], axis = 1)
    df.insert(0, 'Pair', pair)
    return df

In [192]:
start_day = "20170901"
end_day = "20211231"
symbols = ['ETH/USDT']

res = []
for symbol in symbols:
    res.append(ohlcv([start_day, end_day], symbol, '1h'))
    
df = pd.concat(res, axis = 0)

20170901 20211231


In [193]:
df

,Pair,yDay,wDay,Hour,Open,High,Low,Close,Volume
0,ETH/USDT,244,4,9,386.44,388.74,384.79,388.74,162.97779
1,ETH/USDT,244,4,10,388.69,394.06,387.53,391.37,248.72949
2,ETH/USDT,244,4,11,392.88,394.39,389.46,390.88,198.82856
3,ETH/USDT,244,4,12,391.70,393.13,390.86,390.86,141.56114
4,ETH/USDT,244,4,13,392.65,394.27,387.64,390.33,135.65894
...,...,...,...,...,...,...,...,...,...
25307,ETH/USDT,357,3,20,3943.68,3958.81,3921.24,3934.32,10741.62840
25308,ETH/USDT,357,3,21,3934.31,3958.33,3925.01,3944.23,8235.16290
25309,ETH/USDT,357,3,22,3944.23,3965.50,3943.60,3947.53,10538.24940
25310,ETH/USDT,357,3,23,3947.54,3955.68,3925.54,3940.69,15376.04730


data = create_stock(df)

In [194]:
df.to_csv('/Volumes/GoogleDrive/내 드라이브/stock-ai/dataset/ETHUSDT_1h.csv', index = False)

In [33]:
((data['close']-data['open'])/data['open'] * 100).abs().mean()

0.6089043146693095

In [16]:
[(((data['close']-data['close'].shift(n))/data['close'])*100).abs().mean() for n in range (24)]

[0.0,
 0.9787048904000825,
 1.540184717468691,
 2.030538998071487,
 2.4865877568769017,
 2.929156208202759,
 3.3577865846513006,
 3.769365871183702,
 4.179326034157048,
 4.5798074765284085,
 4.9721191852893485,
 5.36517282694626,
 5.752945727540854,
 6.139999265171288,
 6.525956004039234,
 6.914976991212183,
 7.302813768804734,
 7.684520235374529,
 8.061388431713741,
 8.443901872842718,
 8.824117925559811,
 9.201054559679873,
 9.582938733538457,
 9.96628300104373]